# LTFS Data Science FinHack 2

LTFS receives a lot of requests for its various finance offerings that include housing loan, two-wheeler loan, real estate financing and micro loans. The number of applications received is something that varies a lot with season. Going through these applications is a manual process and is tedious. Accurately forecasting the number of cases received can help with resource and manpower management resulting into quick response on applications and more efficient processing.

## Problem Statement
You have been appointed with the task of forecasting daily cases for next 3 months for 2 different business segments aggregated at the country level keeping in consideration the following major Indian festivals (inclusive but not exhaustive list): Diwali, Dussehra, Ganesh Chaturthi, Navratri, Holi etc. (You are free to use any publicly available open source external datasets). Some other examples could be:

Weather Macroeconomic variables Note that the external dataset must belong to a reliable source.

Data Dictionary The train data has been provided in the following way:

* For business segment 1, historical data has been made available at branch ID level For business segment 2, historical data has been made available at State level.

Train File Variable Definition application_date Date of application segment Business Segment (1/2) branch_id Anonymised id for branch at which application was received state State in which application was received (Karnataka, MP etc.) zone Zone of state in which application was received (Central, East etc.) case_count (Target) Number of cases/applications received

Test File Forecasting needs to be done at country level for the dates provided in test set for each segment.

Variable Definition id Unique id for each sample in test set application_date Date of application segment Business Segment (1/2)

### Evaluation
Evaluation Metric The evaluation metric for scoring the forecasts is **MAPE (Mean Absolute Percentage Error)* M with the formula:


Where At is the actual value and Ft is the forecast value.

The Final score is calculated using MAPE for both the segments using the formula:

### Important Notes

Note that feasibility of implementation of top solutions will be considered while adjudging winners The solution must produce satisfactory results for both the business segments

Public and Private Split Test data is further divided into Public (1st Month) and Private (Next 2 months)

# Import Libraries

In [1]:
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ltfs-finhack2/sample_submission_IIzFVsf.csv
/kaggle/input/ltfs-finhack2/train_fwYjLYX.csv
/kaggle/input/ltfs-finhack2/test_1eLl9Yf.csv


In [2]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [3]:
import holidays
from datetime import date

ind_holidays = holidays.IND(years=[2017,2018,2019])

hol=['2017-02-24','2017-03-13','2017-03-28','2017-04-05','2017-04-09','2017-04-14','2017-05-10','2017-06-25','2017-06-26','2017-08-07','2017-09-30',
     '2017-10-02','2017-10-19','2018-02-24','2018-03-13','2018-03-28','2018-04-05','2018-04-09','2018-04-14','2018-05-10','2018-06-25','2018-06-26',
     '2018-08-07','2018-09-30','2018-10-02','2018-10-19','2019-02-24','2019-03-13','2019-03-28','2019-04-05','2019-04-09','2019-04-14','2019-05-10',
     '2019-06-25','2019-06-26','2019-08-07','2019-09-30','2019-10-02','2019-10-19']

for date, name in sorted(ind_holidays.items()):
    hol.append(str(date))

# Import Datasets

In [4]:
train = pd.read_csv('/kaggle/input/ltfs-finhack2/train_fwYjLYX.csv', parse_dates=True)
test = pd.read_csv('/kaggle/input/ltfs-finhack2/test_1eLl9Yf.csv',parse_dates=True)
sample_sub = pd.read_csv('/kaggle/input/ltfs-finhack2/sample_submission_IIzFVsf.csv')

# Data Processing

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80402 entries, 0 to 80401
Data columns (total 6 columns):
application_date    80402 non-null object
segment             80402 non-null int64
branch_id           66898 non-null float64
state               80402 non-null object
zone                66898 non-null object
case_count          80402 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 3.7+ MB


In [6]:
train.shape, test.shape, sample_sub.shape

((80402, 6), (180, 3), (180, 4))

In [7]:
train.nunique()

application_date     844
segment                2
branch_id             83
state                 20
zone                   5
case_count          3245
dtype: int64

In [8]:
train.head()

,application_date,segment,branch_id,state,zone,case_count
0,2017-04-01,1,1.0,WEST BENGAL,EAST,40.0
1,2017-04-03,1,1.0,WEST BENGAL,EAST,5.0
2,2017-04-04,1,1.0,WEST BENGAL,EAST,4.0
3,2017-04-05,1,1.0,WEST BENGAL,EAST,113.0
4,2017-04-07,1,1.0,WEST BENGAL,EAST,76.0


In [9]:
test.tail()

,id,application_date,segment
175,176,2019-10-20,2
176,177,2019-10-21,2
177,178,2019-10-22,2
178,179,2019-10-23,2
179,180,2019-10-24,2


In [10]:
test['segment'].value_counts()

2    93
1    87
Name: segment, dtype: int64

In [11]:
sample_sub.head()

,id,application_date,segment,case_count
0,1,2019-07-06,1,5
1,2,2019-07-07,1,5
2,3,2019-07-08,1,5
3,4,2019-07-09,1,5
4,5,2019-07-10,1,5


In [12]:
train['application_date'] = pd.to_datetime(train['application_date'],format='%Y-%m-%d')
test['application_date'] = pd.to_datetime(test['application_date'],format='%Y-%m-%d')

In [13]:
train1 = train[train['segment'] == 1].groupby(['application_date']).sum().reset_index()[['application_date','case_count']].sort_values('application_date')
train2 = train[train['segment'] == 2].groupby(['application_date']).sum().reset_index()[['application_date','case_count']].sort_values('application_date')

In [14]:
def create_date_featues(df):
    df['Year'] = pd.to_datetime(df['application_date']).dt.year
    df['Month'] = pd.to_datetime(df['application_date']).dt.month
    df['Day'] = pd.to_datetime(df['application_date']).dt.day
    df['Dayofweek'] = pd.to_datetime(df['application_date']).dt.dayofweek
    df['DayOfyear'] = pd.to_datetime(df['application_date']).dt.dayofyear
    df['Week'] = pd.to_datetime(df['application_date']).dt.week 
    df['Quarter'] = pd.to_datetime(df['application_date']).dt.quarter  
    df['Is_month_start'] = pd.to_datetime(df['application_date']).dt.is_month_start 
    df['Is_month_end'] = pd.to_datetime(df['application_date']).dt.is_month_end 
    df['Is_quarter_start'] = pd.to_datetime(df['application_date']).dt.is_quarter_start
    df['Is_quarter_end'] = pd.to_datetime(df['application_date']).dt.is_quarter_end 
    df['Is_year_start'] = pd.to_datetime(df['application_date']).dt.is_year_start 
    df['Is_year_end'] = pd.to_datetime(df['application_date']).dt.is_year_end
    df['Semester'] = np.where(df['Quarter'].isin([1,2]),1,2)
    #df['Is_weekend'] = np.where(df['Dayofweek'].isin([5,6]),1,0)
    #df['Is_weekday'] = np.where(df['Dayofweek'].isin([0,1,2,3,4]),1,0)
    #df['Days_in_month'] = pd.to_datetime(df['application_date']).dt.days_in_month 
    return df

In [15]:
train1 = create_date_featues(train1)
train2 = create_date_featues(train2)

In [16]:
train1['Is_holiday'] = train1['application_date'].isin(hol)
train2['Is_holiday'] = train2['application_date'].isin(hol)

In [17]:
train1.head() 

,application_date,case_count,Year,Month,Day,Dayofweek,DayOfyear,Week,Quarter,Is_month_start,Is_month_end,Is_quarter_start,Is_quarter_end,Is_year_start,Is_year_end,Semester,Is_holiday
0,2017-04-01,299.0,2017,4,1,5,91,13,2,True,False,True,False,False,False,1,False
1,2017-04-03,42.0,2017,4,3,0,93,14,2,False,False,False,False,False,False,1,False
2,2017-04-04,23.0,2017,4,4,1,94,14,2,False,False,False,False,False,False,1,False
3,2017-04-05,1530.0,2017,4,5,2,95,14,2,False,False,False,False,False,False,1,True
4,2017-04-07,1341.0,2017,4,7,4,97,14,2,False,False,False,False,False,False,1,False


In [18]:
train1.drop(['application_date'], axis=1, inplace=True)
train2.drop(['application_date'], axis=1, inplace=True)

In [19]:
train1 = train1[train1['case_count'] > 15].reset_index(drop=True)
train1 = train1[train1['case_count'] <= 8000].reset_index(drop=True)
train2 = train2[(train2['case_count'] < 40000)].reset_index(drop=True)

In [20]:
train1.shape, train2.shape

((780, 16), (843, 16))

# Train test split

## Business Segment 1

In [21]:
X = train1.drop(labels=['case_count'], axis=1)
y = train1['case_count'].values

from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.25, random_state=42)

In [22]:
X_train.shape, y_train.shape, X_cv.shape, y_cv.shape

((585, 15), (585,), (195, 15), (195,))

In [23]:
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
br = RandomForestRegressor(random_state=0)
br.fit(X_train, y_train)
y_pred = br.predict(X_cv)
print('MAPE:', mean_absolute_percentage_error(y_cv, y_pred))

MAPE: 269.4215485248144


## Business Segment 2

In [24]:
X = train2.drop(labels=['case_count'], axis=1)
y = train2['case_count'].values

from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.25, random_state=42)

In [25]:
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
br = RandomForestRegressor(random_state=0)
br.fit(X_train, y_train)
y_pred = br.predict(X_cv)
print('MAPE:', mean_absolute_percentage_error(y_cv, y_pred))

MAPE: 24.816689416412455


# Predict on test set

In [26]:
test.head(2)

,id,application_date,segment
0,1,2019-07-06,1
1,2,2019-07-07,1


In [27]:
test1 = test[test['segment'] == 1][['application_date']].sort_values('application_date')
test2 = test[test['segment'] == 2][['application_date']].sort_values('application_date')

In [28]:
test1 = create_date_featues(test1)
test2 = create_date_featues(test2)

In [29]:
test1['Is_holiday'] = test1['application_date'].isin(hol)
test2['Is_holiday'] = test2['application_date'].isin(hol)

In [30]:
test1.drop(['application_date'], axis=1, inplace=True)
test2.drop(['application_date'], axis=1, inplace=True)

In [31]:
test1.shape, test2.shape

((87, 15), (93, 15))

## Business Segment 1

In [32]:
X = train1.drop(labels=['case_count'], axis=1)
y = train1['case_count'].values
Xtest = test1

In [33]:
X.head()

,Year,Month,Day,Dayofweek,DayOfyear,Week,Quarter,Is_month_start,Is_month_end,Is_quarter_start,Is_quarter_end,Is_year_start,Is_year_end,Semester,Is_holiday
0,2017,4,1,5,91,13,2,True,False,True,False,False,False,1,False
1,2017,4,3,0,93,14,2,False,False,False,False,False,False,1,False
2,2017,4,4,1,94,14,2,False,False,False,False,False,False,1,False
3,2017,4,5,2,95,14,2,False,False,False,False,False,False,1,True
4,2017,4,7,4,97,14,2,False,False,False,False,False,False,1,False


In [34]:
X.shape, y.shape, Xtest.shape

((780, 15), (780,), (87, 15))

In [35]:
from sklearn.model_selection import KFold
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor

rs = 101
errlgb = []
y_pred_totlgb = []

fold = KFold(n_splits=15, shuffle=True, random_state=rs)

for train_index, test_index in fold.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #br = RandomForestRegressor(random_state=rs)
    br = BaggingRegressor(n_estimators=10, random_state=1)
    br.fit(X_train, y_train)
    y_pred = br.predict(X_test)
    print("MAPE: ", mean_absolute_percentage_error(y_test, y_pred))
    errlgb.append(mean_absolute_percentage_error(y_test, y_pred))
    p = br.predict(Xtest)    
    y_pred_totlgb.append(p)

MAPE:  139.54380281136295
MAPE:  63.72317459152785
MAPE:  99.15529425939245
MAPE:  319.77299482206337
MAPE:  85.00920604337232
MAPE:  86.23510678516502
MAPE:  83.43339071301497
MAPE:  74.88667657936216
MAPE:  19.966421459030265
MAPE:  124.8798726368101
MAPE:  88.32137219955723
MAPE:  31.853323886266754
MAPE:  55.127378214518885
MAPE:  222.9748798138905
MAPE:  128.00161862212966


In [36]:
np.mean(errlgb)

108.19230089583097

In [37]:
y_pred1 = np.mean(y_pred_totlgb,0)
test1['case_count'] = y_pred1
test1.head()

,Year,Month,Day,Dayofweek,DayOfyear,Week,Quarter,Is_month_start,Is_month_end,Is_quarter_start,Is_quarter_end,Is_year_start,Is_year_end,Semester,Is_holiday,case_count
0,2019,7,6,5,187,27,3,False,False,False,False,False,False,2,False,2856.900000
1,2019,7,7,6,188,27,3,False,False,False,False,False,False,2,False,1746.833333
2,2019,7,8,0,189,28,3,False,False,False,False,False,False,2,False,4103.873333
3,2019,7,9,1,190,28,3,False,False,False,False,False,False,2,False,3536.120000
4,2019,7,10,2,191,28,3,False,False,False,False,False,False,2,False,3681.453333


In [38]:
test1.drop('case_count', axis=1, inplace=True)

## Business Segment 2

In [39]:
X = train2.drop(labels=['case_count'], axis=1)
y = train2['case_count'].values
Xtest = test2

In [40]:
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor

rs = 101
errlgb = []
y_pred_totlgb = []

fold = KFold(n_splits=15, shuffle=True, random_state=rs)

for train_index, test_index in fold.split(X):
    X_train, X_test = X.loc[train_index], X.loc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #br = RandomForestRegressor(random_state=rs)
    br = BaggingRegressor(n_estimators=10, random_state=rs)
    br.fit(X_train, y_train)
    y_pred = br.predict(X_test)
    print("MAPE: ", mean_absolute_percentage_error(y_test, y_pred))
    errlgb.append(mean_absolute_percentage_error(y_test, y_pred))
    p = br.predict(Xtest)
    y_pred_totlgb.append(p)

MAPE:  28.199633374168513
MAPE:  23.5522591623917
MAPE:  68.09462367527296
MAPE:  20.630346434056847
MAPE:  23.119737125254293
MAPE:  25.182672863971455
MAPE:  19.43303315111579
MAPE:  20.692230037799536
MAPE:  20.838826500175802
MAPE:  14.80553177926803
MAPE:  31.87529544503569
MAPE:  21.950667964336162
MAPE:  18.13027824093041
MAPE:  19.80911990219689
MAPE:  25.75019287878


In [41]:
np.mean(errlgb)

25.470963235650274

In [42]:
y_pred2 = np.mean(y_pred_totlgb,0)
test2['case_count'] = y_pred2

In [43]:
test2.tail()

,Year,Month,Day,Dayofweek,DayOfyear,Week,Quarter,Is_month_start,Is_month_end,Is_quarter_start,Is_quarter_end,Is_year_start,Is_year_end,Semester,Is_holiday,case_count
175,2019,10,20,6,293,42,4,False,False,False,False,False,False,2,False,16146.993333
176,2019,10,21,0,294,43,4,False,False,False,False,False,False,2,False,23640.100000
177,2019,10,22,1,295,43,4,False,False,False,False,False,False,2,False,26812.140000
178,2019,10,23,2,296,43,4,False,False,False,False,False,False,2,False,28211.240000
179,2019,10,24,3,297,43,4,False,False,False,False,False,False,2,False,28125.320000


# Submission

In [44]:
len(y_pred1), len(y_pred2)

(87, 93)

In [45]:
y_pred = np.concatenate((y_pred1, y_pred2))

In [46]:
test['case_count'] = y_pred

In [47]:
test.to_csv('Output.csv', index=False)

In [48]:
test.head(15)

,id,application_date,segment,case_count
0,1,2019-07-06,1,2856.900000
1,2,2019-07-07,1,1746.833333
2,3,2019-07-08,1,4103.873333
3,4,2019-07-09,1,3536.120000
4,5,2019-07-10,1,3681.453333
5,6,2019-07-11,1,3654.306667
6,7,2019-07-12,1,3660.973333
7,8,2019-07-13,1,2791.260000
8,9,2019-07-14,1,1746.260000
9,10,2019-07-15,1,3912.826667


In [49]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

def create_download_link(df, title = "Download CSV file", filename = "submission.csv"):  
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

create_download_link(test)